## Applied Event Detection for Time-Series Data using Space-Filling Curves on the Example of Automotive Data

Event detection is essential for various application domains. In this example, we focus on the automotive domain. We will use space-filling curves (SFCs) to reduce data dimensionality of the original data space for our time-series data. Next, we will work on the compactified, single-dimensional data space to identify patterns that correlate with patterns in the multi-dimensional data space and hence, can be interpreted as events.

### 1. Introducing Space-Filling Curve Computations

First, we use an external library to compute single dimensional representations from multi-dimensional data called [zCurve](https://github.com/rmrschub/zCurve). The library can be used as follows:

In [8]:
import zCurve as z

# Combine the 3D point (2, 16, 8) into a Morton code.
morton_code = z.interlace(2, 16, 8)
assert morton_code == 10248, "morton_code should be 10248"
print( morton_code )

10248


Next, we focus on two-dimensional data as found in GPS positions. Please note that we have transformed floating point, GPS positions to positive integers as SFCs operate on positive input value.

In [9]:
import zCurve as z

morton_code1 = z.interlace(57772400, 12765000, dims=2)
assert morton_code1 == 1606428908008832, "morton_code1 should be 1606428908008832"

morton_code2 = z.interlace(57773800, 12772000, dims=2)
assert morton_code2 == 1606429041286208, "morton_code2 should be 1606429041286208"

print( str(morton_code1) + "\n" + str(morton_code2) )

1606428908008832
1606429041286208


Next, we add a function to turn floating point numbers into positive integers using a dedicated function. The expected values originate from the [corresponding C implementation](https://github.com/chrberger/cluon-cabinet/blob/ZEBRA/test/tests-morton.cpp#L46-L84).

In [10]:
import zCurve as z

def calculateMortonFromTwoLatLonFloats_with_zCurve(x, y):
    # Fix floating point numbers to six decimal places.
    x_int = int( round( (x + 90.0), 5 ) * 100000 )
    y_int = int( round( (y + 180.0), 5 ) * 100000 )
    value = z.interlace(x_int, y_int, dims=2)
    return value

morton_code1_f = calculateMortonFromTwoLatLonFloats_with_zCurve(60.734398, 14.768745)
assert morton_code1_f == 664749436224648, "morton_code1_f should be 664749436224648"
print( "Two positive floats: " + str(morton_code1_f) )

morton_code2_f = calculateMortonFromTwoLatLonFloats_with_zCurve(38.969745, -77.201958)
assert morton_code2_f == 231657429695220, "morton_code2_f should be 231657429695220"
print( "One positive/one negative float: " + str(morton_code2_f) )

morton_code3_f = calculateMortonFromTwoLatLonFloats_with_zCurve(-34.619055, -58.364067)
assert morton_code3_f == 171054631290070, "morton_code3_f should be 171054631290070"
print( "Two negative floats: " + str(morton_code3_f) )

morton_code4_f = calculateMortonFromTwoLatLonFloats_with_zCurve(-33.956603, 150.949719)
assert morton_code4_f == 769185334910896, "morton_code4_f should be 769185334910896"
print( "One negative/one positive float floats: " + str(morton_code4_f) )

Two positive floats: 664749436224648
One positive/one negative float: 231657429695220
Two negative floats: 171054631290070
One negative/one positive float floats: 769185334910896


As we want identical results on Python and C++, we supply an own implementation to calculate Morton codes from two-dimensional values. The corresponding C++ implementation can be found [here](https://github.com/chrberger/cluon-cabinet/blob/ZEBRA/src/morton.hpp#L41-L74) and the original source [here (CC BY-SA 4.0)](https://stackoverflow.com/a/30562230). Now, it is important that we are precise about the underlying dataypes and hence, we are using NumPy.

In [11]:
import numpy as np

def mortonEncode2D(a, b):
    x = np.uint64(a)
    y = np.uint64(b)

    x = (x | (x << np.uint64(16))) & np.uint64(0x0000FFFF0000FFFF)
    x = (x | (x << np.uint64(8)))  & np.uint64(0x00FF00FF00FF00FF)
    x = (x | (x << np.uint64(4)))  & np.uint64(0x0F0F0F0F0F0F0F0F)
    x = (x | (x << np.uint64(2)))  & np.uint64(0x3333333333333333)
    x = (x | (x << np.uint64(1)))  & np.uint64(0x5555555555555555)

    y = (y | (y << np.uint64(16))) & np.uint64(0x0000FFFF0000FFFF)
    y = (y | (y << np.uint64(8)))  & np.uint64(0x00FF00FF00FF00FF)
    y = (y | (y << np.uint64(4)))  & np.uint64(0x0F0F0F0F0F0F0F0F)
    y = (y | (y << np.uint64(2)))  & np.uint64(0x3333333333333333)
    y = (y | (y << np.uint64(1)))  & np.uint64(0x5555555555555555)

    result = np.uint64( x | (y << np.uint64(1)) )
    return result

def mortonExtractEvenBits(a):
    x = np.uint64(a)
    
    x = x & np.uint64(0x5555555555555555)
    x = (x | (x >> np.uint64(1)))  & np.uint64(0x3333333333333333)
    x = (x | (x >> np.uint64(2)))  & np.uint64(0x0F0F0F0F0F0F0F0F)
    x = (x | (x >> np.uint64(4)))  & np.uint64(0x00FF00FF00FF00FF)
    x = (x | (x >> np.uint64(8)))  & np.uint64(0x0000FFFF0000FFFF)
    x = (x | (x >> np.uint64(16))) & np.uint64(0x00000000FFFFFFFF)
    
    return x.astype(np.uint32)

def mortonDecode2D(a):
    _a = np.uint64(a)
    
    x = mortonExtractEvenBits(_a)
    y = mortonExtractEvenBits(_a >> np.uint64(1))
    
    return (x, y)

morton_code1 = mortonEncode2D(57772400, 12765000)
assert morton_code1 == 1606428908008832, "morton_code1 should be 1606428908008832"
print( morton_code1 )

decode_morton_code1 = mortonDecode2D(1606428908008832)
assert decode_morton_code1 == (57772400, 12765000), "decode_morton_code1 should be (57772400, 12765000)"
print( decode_morton_code1 )

1606428908008832
(57772400, 12765000)


Finally, we can compare the two implementations.

In [12]:
morton_code1_3rd = z.interlace(57772400, 12765000, dims=2)
morton_code1_own = mortonEncode2D(57772400, 12765000)

print( morton_code1_3rd - morton_code1_own )

0.0


Now, we are putting every piece together.

In [13]:
def calculateMortonFromTwoLatLonFloats(x, y):
    _x = np.float32(x)
    _y = np.float32(y)
    # Fix floating point numbers to six decimal places.
    x_int = np.uint32( np.round( (_x + np.float32(90.0) ), 6 ) * np.uint32(100000) )
    y_int = np.uint32( np.round( (_y + np.float32(180.0) ), 6 ) * np.uint32(100000) )
    value = mortonEncode2D(x_int, y_int)
    return value

p1 = (60.734398, 14.768745)
morton_code1 = calculateMortonFromTwoLatLonFloats(p1[0], p1[1])
assert morton_code1 == 664749436224642, "morton_code1 should be 664749436224642"
print( "Two positive floats: " + str(morton_code1) )
print( "Delta to 3rd party library: " + str(morton_code1 - morton_code1_f) )

p2 = (38.969745, -77.201958)
morton_code2 = calculateMortonFromTwoLatLonFloats(p2[0], p2[1])
assert morton_code2 == 231657429695220, "morton_code2 should be 231657429695220"
print( "One positive/one negative float: " + str(morton_code2) )
print( "Delta to 3rd party library: " + str(morton_code2 - morton_code2_f) )

p3 = (-34.619055, -58.364067)
morton_code3 = calculateMortonFromTwoLatLonFloats(p3[0], p3[1])
assert morton_code3 == 171054631290070, "morton_code3 should be 171054631290070"
print( "Two negative floats: " + str(morton_code3) )
print( "Delta to 3rd party library: " + str(morton_code3 - morton_code3_f) )

p4 = (-33.956603, 150.949719)
morton_code4 = calculateMortonFromTwoLatLonFloats(p4[0], p4[1])
assert morton_code4 == 769185334910861, "morton_code4 should be 769185334910861"
print( "One negative/one positive float floats: " + str(morton_code4) )
print( "Delta to 3rd party library: " + str(morton_code4 - morton_code4_f) )

Two positive floats: 664749436224642
Delta to 3rd party library: -6.0
One positive/one negative float: 231657429695220
Delta to 3rd party library: 0.0
Two negative floats: 171054631290070
Delta to 3rd party library: 0.0
One negative/one positive float floats: 769185334910861
Delta to 3rd party library: -35.0


Finally, we convert the Morton code back into two floating point numbers.

In [14]:
import sys

def calculateTwoLatLonFloatsFromMorton(a):
    pair = mortonDecode2D(a)
    _x = np.float32(pair[0] / np.float32(100000.0) - np.float32(90.0))
    _y = np.float32(pair[1] / np.float32(100000.0) - np.float32(180.0))
    return (_x, _y)

eps = 0.00001
decode_morton_code1 = calculateTwoLatLonFloatsFromMorton(morton_code1)
print( "{:.6f}".format(decode_morton_code1[0]) + " " + "{:.6f}".format(decode_morton_code1[1]))
print( "Delta: " + "{:.6f}".format(decode_morton_code1[0] - p1[0]) + " " + "{:.6f}".format(decode_morton_code1[1] - p1[1]))
assert np.fabs(decode_morton_code1[0] - p1[0]) < eps , "Delta for p1 after decoding too large"

decode_morton_code2 = calculateTwoLatLonFloatsFromMorton(morton_code2)
print( "{:.6f}".format(decode_morton_code2[0]) + " " + "{:.6f}".format(decode_morton_code2[1]))
print( "Delta: " + "{:.6f}".format(decode_morton_code2[0] - p2[0]) + " " + "{:.6f}".format(decode_morton_code2[1] - p2[1]))
assert np.fabs(decode_morton_code2[0] - p2[0]) < eps , "Delta for p2 after decoding too large"

decode_morton_code3 = calculateTwoLatLonFloatsFromMorton(morton_code3)
print( "{:.6f}".format(decode_morton_code3[0]) + " " + "{:.6f}".format(decode_morton_code3[1]))
print( "Delta: " + "{:.6f}".format(decode_morton_code3[0] - p3[0]) + " " + "{:.6f}".format(decode_morton_code3[1] - p3[1]))
assert np.fabs(decode_morton_code3[0] - p3[0]) < eps , "Delta for p3 after decoding too large"

decode_morton_code4 = calculateTwoLatLonFloatsFromMorton(morton_code4)
print( "{:.6f}".format(decode_morton_code4[0]) + " " + "{:.6f}".format(decode_morton_code4[1]))
print( "Delta: " + "{:.6f}".format(decode_morton_code4[0] - p4[0]) + " " + "{:.6f}".format(decode_morton_code4[1] - p4[1]))
assert np.fabs(decode_morton_code4[0] - p4[0]) < eps , "Delta for p4 after decoding too large"

60.734402 14.768730
Delta: 0.000004 -0.000015
38.969742 -77.201958
Delta: -0.000003 0.000000
-34.619061 -58.364071
Delta: -0.000006 -0.000004
-33.956612 150.949707
Delta: -0.000009 -0.000012
